# Energy Consumption data analysis

All this data was pulled from the US Energy Information Administration (EIA). In this notebook i imported most of the datasets from the link below and linked joined them together into one dataset. The final dataset consists of the columns:

['Country', 'Energy_type', 'Year', 'Energy_consumption', 'Energy_production', 'GDP', 'Population', 'energy intensity by GDP and population', 'C02_emissions']

https://www.eia.gov/international/data/world?pd=10&p=00000000000000000000000000000000000000000000000c000g&u=0&f=A&v=mapbubble&a=-&i=none&vo=value&t=C&g=00000000000000000000000000000000000000000000000001&l=249-ruvvvvvfvtvnvv1vrvvvvfvvvvvvfvvvou20evvvvvvvvvvnvvvs0008&s=315532800000&e=1388534400000

In [83]:
# Analysis Tools
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm

# Plotting Design Settings
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": "#E2E2E2", 'grid.color': '#B0B0B0', 'patch.edgecolor': '#B0B0B0', 'font.sans-serif': 'Verdana'})
sns.set_palette('twilight')

# Feature Engineering
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder, OrdinalEncoder
from feature_engine.outliers import Winsorizer
from feature_engine.selection import DropConstantFeatures
from collections import defaultdict
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Modeling
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


# Scoring
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


# Mute warnings
import warnings
warnings.filterwarnings('ignore')

## Importing dataset and making it usable

In [84]:
# Creating column names for both datasets
year_columns = ['API', 'Country', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
                '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
                '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


# Check periodicly if new dataset with 2020 data shows up!!!!!!

# Total Energy Consumption dataset
econ = pd.read_csv('C:/Users/benva/OneDrive/Documents/Code/Machine Learning/GlobalWarmingProject/Datasets/Total_energy_consumption.csv', names=year_columns)

# Total Energy Production dataset
eprod = pd.read_csv('C:/Users/benva/OneDrive/Documents/Code/Machine Learning/GlobalWarmingProject/Datasets/Total_energy_production.csv', names=year_columns)

# GDP by year of each Country dataset
gdp = pd.read_csv('C:/Users/benva/OneDrive/Documents/Code/Machine Learning/GlobalWarmingProject/Datasets/GDP_by_country.csv', names=year_columns)

# Population by year of each Country Dataset
pop = pd.read_csv('C:/Users/benva/OneDrive/Documents/Code/Machine Learning/GlobalWarmingProject/Datasets/Population_by_country.csv', names=year_columns)

# Energy intensity by GDP and population
ei_gdp_pop = pd.read_csv('C:/Users/benva/OneDrive/Documents/Code/Machine Learning/GlobalWarmingProject/Datasets/Energy_intensity_by_GDP_and_population.csv', names=year_columns)



# this one goes from 1949 - 2019
co2_year_columns = ['API', 'Country', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1961', '1962',
                '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
                '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
                '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
                '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

# C02 Emissions
co2_e = pd.read_csv('C:/Users/benva/OneDrive/Documents/Code/Machine Learning/GlobalWarmingProject/Datasets/Emissions_by_fuel.csv', names=co2_year_columns)

In [85]:
econ.shape

(1626, 42)

In [86]:
eprod.shape

(1626, 42)

In [87]:
gdp.shape

(234, 42)

In [88]:
pop.shape

(234, 42)

In [89]:
ei_gdp_pop.shape

(466, 42)

In [90]:
co2_e.shape

(930, 72)

In [91]:
econ.head()

,API,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Report generated on: 12-29-2021 21:17:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,API,NaN,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2,NaN,total energy consumption (quad Btu),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTL.44-2-WORL-QBTU.A,World,292.8999360117116,289.40151331082484,289.69127227568276,293.7314691613172,307.2346122721334,315.42743540651674,322.18517006014446,332.74655318366644,...,527.0687498431604,540.8952031029451,551.9303162477122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,INTL.44-2-AFG-QBTU.A,Afghanistan,0.02658321667756184,0.029949457623843767,0.031897814673808554,0.03902180313163165,0.03920224040186807,0.03827496129519434,0.039111198580053776,0.06306515769145625,...,0.11853852864995157,0.16105080218714038,0.1476076485691699,0.12450427328434109,0.1143137511302674,0.12629013628746977,0.11757565429923286,0.12494125714475693,0.14327311946093876,0.12274842133951257


In [92]:
eprod.head()

,API,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Report generated on: 01-01-2022 16:23:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,API,NaN,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2,NaN,total energy production (quad Btu),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTL.44-1-WORL-QBTU.A,World,296.33722758863627,291.2877731828766,290.1205665759991,293.0466469371524,308.92101366220777,316.46426154850144,326.7530652183027,334.3845057886132,...,518.2107838608919,533.8925086831691,545.3963111029857,553.2388229380201,561.949952486594,567.4371747031059,563.3786478855133,577.8623821917073,600.076613912069,610.9984170870961
4,INTL.44-1-AFG-QBTU.A,Afghanistan,0.07256115614667066,0.09272365463746368,0.09506786743187949,0.09971281973566948,0.09982734303917455,0.12096783009943252,0.12054976892110061,0.114835081155632,...,0.02692579367150454,0.04104421211242948,0.037492262916033615,0.040816563280627896,0.04478950651933605,0.04205799729038833,0.04950511782484147,0.05950054255404184,0.06317585556177092,0.05521030952760256


Both the datasets above, econ and eprod are identical, other than the fact that econ is on energy consumption and eprod is on energy production

In [93]:
gdp.head()

,API,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Report generated on: 01-01-2022 16:51:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,API,NaN,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2,NaN,GDP at purchasing power parities (Billion 2015...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTL.4701-34-WORL-BDOLPPP.A,World,27771.471791,28666.385038,28802.329207,29618.135692,31097.825221,32289.628281,33425.965914,34793.408835,...,94618.286635,98369.8792,101534.904378,104859.866766,108405.189058,111936.011938,115568.027605,119855.194807,124162.239335,127689.100659
4,INTL.4701-34-AFG-BDOLPPP.A,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,56.7712,60.6103,66.4914,70.9136,73.0931,72.056,73.2453,75.1899,76.3784,79.4546


In [94]:
pop.head()

,API,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Report generated on: 01-01-2022 16:59:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,API,NaN,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2,NaN,population (Mperson),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTL.4702-33-WORL-THP.A,World,4298126.5224,4377059.5593,4456829.97443,4537793.89383,4572800.78237,4657357.404,4745135.92538,4835605.8458,...,6948810.014,7041712.689,7126262.29402,7211822.209,7297269.981,7379227.32003,7464042.84623,7548343.78937,7632247.01155,7714631.04083
4,INTL.4702-33-AFG-THP.A,Afghanistan,13356.5,13171.7,12882.5,12537.7,12204.3,11938.2,11736.2,11604.5,...,29185.5,30117.4,31161.4,32269.6,33370.8,34413.6,35383,36296.1,37173.5,38050.9


In [95]:
ei_gdp_pop.head()

,API,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Report generated on: 01-02-2022 15:08:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,API,NaN,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2,NaN,energy consumption per capita (MMBtu/person),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTL.47-33-WORL-MBTUPP.A,World,68.1459548678342,66.11779195371682,64.99939955926463,64.73001551716604,67.18740371473154,67.72669736181508,67.89798545851839,68.81176088259464,...,75.85021734387001,76.81301793921361,77.45018264495602,77.17074501779302,76.90143146117923,76.24857246626453,76.04634169493572,76.74242174383677,77.59177446964536,77.21384657054325
4,INTL.47-33-AFG-MBTUPP.A,Afghanistan,1.9902831338720353,2.2737731366371663,2.476057805069556,3.11235738067043,3.212166236643484,3.206091479049969,3.332526591235134,5.434543297122344,...,4.061555520719247,5.347433782037638,4.736874741480483,3.8582527606273733,3.4255622019929817,3.6697740511736567,3.32294192971859,3.4422777418168047,3.8541735230994862,3.225900605229116


In [96]:
co2_e.head()

,API,Country,1949,1950,1951,1952,1953,1954,1955,1956,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Report generated on: 01-02-2022 15:04:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,API,NaN,1949.000000,1950.000000,1951.000000,1952.000000,1953.000000,1954.00000,1955.000000,1956.000000,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2,NaN,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTL.4008-8-WORL-MMTCD.A,World,2206.690829,2382.046176,2526.687327,2473.373964,2536.892888,2422.25256,2684.786146,2777.042758,...,31639.876165687,33609.984153055,34720.267922662,35537.365120578,35944.451742368,35976.723878385,35781.370769103,35117.963601548,35887.898987724,36638.369585101
4,INTL.4008-8-AFG-MMTCD.A,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.230759779,7.794436305,11.097938929,9.753422695,7.910700222,7.169380063,7.852509211,7.279395964,7.886124149,9.391163087


#### Removing unecesary cells in each dataset

In [97]:
# Dropping first 2 rows
econ.drop([0, 1], axis=0, inplace=True)
eprod.drop([0, 1], axis=0, inplace=True)
ei_gdp_pop.drop([0, 1], axis=0, inplace=True)
co2_e.drop([0, 1], axis=0, inplace=True)

# Dropping first 3 rows for gdp and pop
gdp.drop([0, 1, 2], axis=0, inplace=True)
pop.drop([0, 1, 2], axis=0, inplace=True)

# Dropping first column 'API'
econ.drop('API', axis=1, inplace=True)
eprod.drop('API', axis=1, inplace=True)
gdp.drop('API', axis=1, inplace=True)
pop.drop('API', axis=1, inplace=True)
ei_gdp_pop.drop('API', axis=1, inplace=True)

# Dropping first column 'API' as well as all columns up to 1980
co2_e.drop(['API', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956',
            '1957', '1958', '1959', '1961', '1962', '1963', '1964', '1965', '1966',
            '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
            '1976', '1977', '1978', '1979'], axis=1, inplace=True)

# Reseting the Index
econ.reset_index(inplace=True, drop=True)
eprod.reset_index(inplace=True, drop=True)
gdp.reset_index(inplace=True, drop=True)
pop.reset_index(inplace=True, drop=True)
ei_gdp_pop.reset_index(inplace=True, drop=True)
co2_e.reset_index(inplace=True, drop=True)

In [98]:
econ

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,total energy consumption (quad Btu),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,World,292.8999360117116,289.40151331082484,289.69127227568276,293.7314691613172,307.2346122721334,315.42743540651674,322.18517006014446,332.74655318366644,345.56128730345165,...,527.0687498431604,540.8952031029451,551.9303162477122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,0.02658321667756184,0.029949457623843767,0.031897814673808554,0.03902180313163165,0.03920224040186807,0.03827496129519434,0.039111198580053776,0.06306515769145625,0.11122078650562646,...,0.11853852864995157,0.16105080218714038,0.1476076485691699,0.12450427328434109,0.1143137511302674,0.12629013628746977,0.11757565429923286,0.12494125714475693,0.14327311946093876,0.12274842133951257
3,Albania,0.16298182182124926,0.16204449273682853,0.16284611700217974,0.16933585149179003,0.1704057218202433,0.17005625500667076,0.16607537777033485,0.1666405283961216,0.1685908728103857,...,0.12304402332284661,0.10743348644120246,0.1051279333181231,0.13193111766441915,0.11621639284070061,0.11629730243544255,0.1296073826057356,0.12535309157946248,0.14218447431148448,0.11074824093269141
4,Algeria,0.7806951671133022,0.6633913232329025,0.952188115918025,1.0705618432163024,1.1307867129706235,1.0464182466677772,1.0663009617019872,1.1383186539191739,1.2131193653810644,...,1.7777604019160995,1.9379902738228294,2.0982041249829453,2.037585597537122,2.299898896758712,2.351782231770593,2.3361518937806958,2.361200900056517,2.5157731727516484,2.6435026580305645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Wake Island,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,Western Sahara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,Yemen,0,0,0,0,0,0,0,0,0,...,0.000019512,0.000019432,0.000028548,0.000038164,0.00008559,0.00102509,0.001357104,0.001685979,0.004169632,0.004452
1622,Zambia,0.0842504416429,0.09095160935112,0.09276673989719,0.093613620726,0.091707448,0.092414884,0.09076501,0.081980147,0.080546524,...,0.099506171504,0.111397717312,0.1142021,0.122629646264,0.12864310304,0.119627497176,0.106216508816,0.113591182889,0.120469815456,0.12081104372


In [99]:
eprod

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,total energy production (quad Btu),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,World,296.33722758863627,291.2877731828766,290.1205665759991,293.0466469371524,308.92101366220777,316.46426154850144,326.7530652183027,334.3845057886132,347.41286310379365,...,518.2107838608919,533.8925086831691,545.3963111029857,553.2388229380201,561.949952486594,567.4371747031059,563.3786478855133,577.8623821917073,600.076613912069,610.9984170870961
2,Afghanistan,0.07256115614667066,0.09272365463746368,0.09506786743187949,0.09971281973566948,0.09982734303917455,0.12096783009943252,0.12054976892110061,0.114835081155632,0.1213943959239381,...,0.02692579367150454,0.04104421211242948,0.037492262916033615,0.040816563280627896,0.04478950651933605,0.04205799729038833,0.04950511782484147,0.05950054255404184,0.06317585556177092,0.05521030952760256
3,Albania,0.155561620237156,0.15869138293903517,0.20594415566567068,0.23469322168485068,0.19102506227402413,0.19172993827692245,0.19659552252877113,0.20238806446252838,0.18641741777005813,...,0.09849116932673144,0.07585689950134678,0.0840378737211778,0.10563648550534094,0.09296194121961915,0.10225470403895913,0.12535621607056832,0.08229649004849868,0.11552948572255144,0.08908229857345225
4,Algeria,2.803017355026457,3.0375368604701833,3.224933778884667,3.6064004828010217,3.8591760033483915,3.907465537569512,3.9683247789434253,4.21808633368004,4.297753784176673,...,6.927837879172395,6.828492246382395,6.999409831111563,6.5104186447871735,6.561760333289481,6.696192522564449,6.910709728136037,6.763856727052585,6.688552187874529,6.429790182459667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Wake Island,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,Western Sahara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,Yemen,0,0,0,0,0,0,0,0,0,...,0.000019512,0.000019432,0.000028548,0.000038164,0.00008559,0.00102509,0.001357104,0.001685979,0.004169632,0.004452
1622,Zambia,0.0935959096429,0.10114325335112,0.10363054789719,0.104463780726,0.10213452,0.102923844,0.10127397,0.087029907,0.085596284,...,0.101433951504,0.111087225312,0.116989704,0.126075766264,0.13288421904,0.120961589176,0.101470416816,0.114648902889,0.124130891456,0.12346557972


In [100]:
gdp

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,World,27771.471791,28666.385038,28802.329207,29618.135692,31097.825221,32289.628281,33425.965914,34793.408835,42107.304103,...,94618.286635,98369.8792,101534.904378,104859.866766,108405.189058,111936.011938,115568.027605,119855.194807,124162.239335,127689.100659
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.7013,...,56.7712,60.6103,66.4914,70.9136,73.0931,72.056,73.2453,75.1899,76.3784,79.4546
2,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.6877,...,30.743,31.5255,31.9724,32.2927,32.8658,33.595,34.7087,36.0284,37.4765,38.2684
3,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219.921,...,405.452,417.21,431.395,443.474,460.326,477.358,492.633,499.038,505.026,509.066
4,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.576,0.574,0.644,0.641,0.642,0.661,0.652,0.602,0.636,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Wake Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,122.668,104.159,106.437,110.258,98.5626,68.543,58.5318,52.7009,51.992,53.0838
229,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.2451,...,169.5431,178.9776,192.5756,202.3145,211.7024,217.9309,226.1135,234.0865,243.5819,247.1819


In [101]:
pop

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,World,4298126.5224,4377059.5593,4456829.97443,4537793.89383,4572800.78237,4657357.404,4745135.92538,4835605.8458,4927545.0842,...,6948810.014,7041712.689,7126262.29402,7211822.209,7297269.981,7379227.32003,7464042.84623,7548343.78937,7632247.01155,7714631.04083
1,Afghanistan,13356.5,13171.7,12882.5,12537.7,12204.3,11938.2,11736.2,11604.5,11618,...,29185.5,30117.4,31161.4,32269.6,33370.8,34413.6,35383,36296.1,37173.5,38050.9
2,Albania,2682.7,2737.02,2790.28,2844.87,2904.14,2969.67,3044.2,3124.89,3200.97,...,2948.03,2928.6,2914.09,2903.79,2896.31,2890.52,2886.43,2884.17,2882.05,2879.92
3,Algeria,19221.7,19824.3,20452.9,21101.9,21763.6,22431.5,23102.4,23774.3,24443.5,...,35977.4,36661.4,37383.9,38140.1,38923.7,39728,40551.4,41389.2,42209.8,43030.4
4,American Samoa,32.646,33.701,34.968,36.412,37.946,39.519,41.119,42.74,44.343,...,56.079,55.759,55.667,55.713,55.791,55.812,55.741,55.62,55.465,55.312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Wake Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,Yemen,7941.9,8231.91,8541.61,8869.36,9213.08,9572.17,9941.1,10322,10730.9,...,23154.8,23807.6,24473.2,25147.1,25823.5,26497.9,27168.2,27834.8,28498.5,29162.2
229,Zambia,5851.817,6055.363,6265.87,6481.905,6701.548,6923.147,7146.965,7372.835,7598.27,...,13606,14023.18,14465.15,14926.53,15399.8,15879.38,16363.45,16853.6,17363.72,17873.85


In [102]:
ei_gdp_pop

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,energy consumption per capita (MMBtu/person),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,World,68.1459548678342,66.11779195371682,64.99939955926463,64.73001551716604,67.18740371473154,67.72669736181508,67.89798545851839,68.81176088259464,70.12848820226563,...,75.85021734387001,76.81301793921361,77.45018264495602,77.17074501779302,76.90143146117923,76.24857246626453,76.04634169493572,76.74242174383677,77.59177446964536,77.21384657054325
2,Afghanistan,1.9902831338720353,2.2737731366371663,2.476057805069556,3.11235738067043,3.212166236643484,3.206091479049969,3.332526591235134,5.434543297122344,9.573143958136207,...,4.061555520719247,5.347433782037638,4.736874741480483,3.8582527606273733,3.4255622019929817,3.6697740511736567,3.32294192971859,3.4422777418168047,3.8541735230994862,3.225900605229116
3,Albania,60.752906333637476,59.20471634727863,58.361926760819614,59.52323005683565,58.67682750151277,57.26436102552498,54.554686870223655,53.326846191744856,52.668682558844885,...,41.73771071625683,36.684247231169316,36.075733185359105,45.43411116658544,40.12567468285529,40.2340417763733,44.90231275511119,43.462449016341786,49.33449257004024,38.4553185271436
4,Algeria,40.61530286672366,33.46354339032917,46.5551641047492,50.73295974373409,51.95770520367143,46.64949943908242,46.155419424041966,47.88021745831313,49.629527906439925,...,49.413253929302826,52.861873082392634,56.125875710745674,53.42370884022648,59.08736571185966,59.19709604738706,57.609648342121254,57.04872044051388,59.60163688886581,61.433374033951914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,Wake Island,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,Western Sahara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
461,Yemen,0,0,0,0,0,0,0,0,0,...,2.7742874485759597,2.745226716195416,2.3278365292917402,3.171370317413033,3.3581897449322806,2.6071051164501315,2.443451042807191,2.504412320318868,2.361163979796315,2.9956346213705087
462,Zambia,0,0,0,0,0,0,0,0,1.7070595067346914,...,0.7683591714822431,0.8154172322799056,0.8142557261121618,0.8218490959250522,0.8429966596642897,0.7924680967724622,0.7203060396079056,0.8025461644402722,0.867474607972482,0.8378305890652841


In [103]:
co2_e

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,World,4946.62713,18701.988660053,18327.455974396,18284.018922293,18478.503517135,20118.811365854,20531.213799098,21064.157108403,21736.103906879,...,31639.876165687,33609.984153055,34720.267922662,35537.365120578,35944.451742368,35976.723878385,35781.370769103,35117.963601548,35887.898987724,36638.369585101
2,Afghanistan,NaN,1.325964518,1.509627735,1.65960468,2.05960468,2.065436114,2.093515917,2.311010238,3.646724197,...,6.230759779,7.794436305,11.097938929,9.753422695,7.910700222,7.169380063,7.852509211,7.279395964,7.886124149,9.391163087
3,Albania,NaN,9.618526399,9.533068754,9.594264089,9.981384922,10.126808525,10.0889756,10.154939541,9.828091766,...,3.02339894,3.673558222,3.756730087,3.393630544,3.857129766,4.043419507,3.913785734,4.105771205,4.483725681,4.031246029
4,Algeria,NaN,45.663590904,39.376193382,56.531989464,63.168954306,66.766862302,61.757201027,60.918466391,64.343004942,...,99.380185852,101.352896736,112.488751213,122.016837732,118.720563943,131.695938497,138.622437629,137.190705524,137.678348102,145.987969135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,Wake Island,NaN,1.23566,1.2,1.2,1.2,1.2,1.2,1.3,1.3,...,1.3,1.2954475286034979,1.2978968706239917,1.3014527524613186,1.2978968706239917,1.2978968706239917,1.2978968706239917,1.298108468455726,1.2719549586239927,1.2737868546239928
924,Western Sahara,NaN,0.15699,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.3,0.3,0.2676640063021204,0.26080606525442773,0.2689223903567869,0.2684851998757719,0.270403,0.269783,0.26303823989786146,0.26303823989786146
925,Yemen,NaN,6.43422,6.6,7.7,7,8.9,9.9,10,9.9,...,20,22,18.55858838418988,15.661195605698236,21.455366,20.888499,9.957683,8.371222,8.15101,7.888358
926,Zambia,NaN,2.21293,2,2,2,1.8,1.7,1.8,1.9,...,2,2,2.2833131328989174,2.6941848042448058,2.8296932137810265,3.2481110278628478,3.551921,3.383059895344,3.750432460396,4.20218759386


#### Energy type name and index locations in both econ and eprod datasets
total energy consumption (quad Btu) - 0
coal (quad Btu) - 232
natural gas (quad Btu) - 464
petroleum and other liquids (quad Btu) - 696
nuclear, renewables, and other (quad Btu) - 928
nuclear (quad Btu) - 1160
renewables and other (quad Btu) - 1392


#### GDP type name and index locations in gdp dataset
GDP at purchasing power parities (Billion 2015$$ PPP) - 0


#### Population type name and index locations in pol dataset
population (Mperson) - 0


#### Energy intensity type name and index locations in ie_gdp_pop dataset
energy consumption per capita (MMBtu/person) - 0
energy consumption per GDP (1000 Btu/2015$ GDP PPP) - 232


#### Emission type name and index location in the co2_e dataset
CO2 emissions (MMtonnes CO2) - 0


theres ['nuclear, renewables, and other'], and also ['nuclear'] with ['renewables and other']. This is kindove redundant, im just going to remove ['nuclear and renewables, and other']

#### Splitting each energy source by index and turning them into seperate temporary datasets for econ, eprod and ei_gdp_pop datasets

In [104]:
# Locating the index position of each Energy Type in econ dataset
total_con = econ.iloc[1:232]
coal_con = econ.iloc[233:464]
natural_gas_con = econ.iloc[465:696]
petroleum_n_other_liquids_con = econ.iloc[697:928]
nuclear_renewables_n_other_con = econ.iloc[929:1160]    #Leaving this one out!!!
nuclear_con = econ.iloc[1161:1392]
renewables_n_other_con = econ.iloc[1393:1624]


# Locating the index position of each Energy Type in eprod dataset
total_prod = eprod.iloc[1:232]
coal_prod = eprod.iloc[233:464]
natural_gas_prod = eprod.iloc[465:696]
petroleum_n_other_liquids_prod = eprod.iloc[697:928]
nuclear_renewables_n_other_prod = eprod.iloc[929:1160]    #Leaving this one out!!!
nuclear_prod = eprod.iloc[1161:1392]
renewables_n_other_prod = eprod.iloc[1393:1624]

# Locating the index position of both Energy intensity of GDP and Population in ei_gdp_pop dataset
energy_consumption_per_capita = ei_gdp_pop[1:232]
energy_consumption_per_GDP = ei_gdp_pop[233:464]

# Locating the index position for CO2 emissions in co2 dataset -- Only using the total emissions for this datset
CO2_emissions = co2_e[1:232]

# Putting all of them into a list
econ_indexes = [total_con, coal_con, natural_gas_con, petroleum_n_other_liquids_con, nuclear_con, renewables_n_other_con]
eprod_indexes = [total_prod, coal_prod, natural_gas_prod, petroleum_n_other_liquids_prod, nuclear_prod, renewables_n_other_prod]
ei_gdp_pop_indexes = [energy_consumption_per_capita, energy_consumption_per_GDP]
co2_e_indexes = [CO2_emissions]

# Function to split into temporary datasets
def temp_df(index):
    # Running the list through a for loop that turns each of them into a seperate dataframe
    for x in index:
        df_name = pd.DataFrame(x)


temp_df(econ_indexes)
temp_df(eprod_indexes)

# Splitting these now and wont be joining it back up, will make these seperate columns at the end
temp_df(ei_gdp_pop_indexes)
temp_df(co2_e_indexes)

#### Creating new column for the specific energy source of each temporary dataset

In [105]:
def make_columns(df, value):
    df.insert(1, 'Energy_type', value)


# for econ dataset
make_columns(total_con, 'all_energy_types')
make_columns(coal_con, 'coal')
make_columns(natural_gas_con, 'natural_gas')
make_columns(petroleum_n_other_liquids_con, 'petroleum_n_other_liquids')
make_columns(nuclear_con, 'nuclear')
make_columns(renewables_n_other_con, 'renewables_n_other')

# for eprod dataset
make_columns(total_prod, 'all_energy_types')
make_columns(coal_prod, 'coal')
make_columns(natural_gas_prod, 'natural_gas')
make_columns(petroleum_n_other_liquids_prod, 'petroleum_n_other_liquids')
make_columns(nuclear_prod, 'nuclear')
make_columns(renewables_n_other_prod, 'renewables_n_other')

# for ei_gdp_pop dataset
make_columns(energy_consumption_per_capita, 'energy_consumption_per_capita')
make_columns(energy_consumption_per_GDP, 'energy_consumption_per_GDP')

# for co2_e datset
make_columns(CO2_emissions, 'CO2_emissions')

In [106]:
coal_con

,Country,Energy_type,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
233,World,coal,78.65628042083658,78.96407276869913,80.42250151969823,82.63101739438098,86.35626815849287,89.40956303455617,90.43265631556591,94.05082968532112,...,158.12448895223338,166.7014764934442,171.73669392982674,173.61828745081039,172.42014067233737,167.89940511651292,163.76411477567194,164.4690218347296,165.86160315821368,164.2084676072012
234,Afghanistan,coal,0.0024792482148388983,0.0026042523630414455,0.0030209327940489256,0.0030209327940489256,0.0030834347736486832,0.003145936942251473,0.0033334430700537776,0.0034792812114562443,...,0.015104663781241597,0.03083430993588047,0.025834151567899852,0.02806338884029121,0.031613501281557446,0.028438400717889757,0.03537612045346286,0.04564727910102304,0.05093911337380253,0.04456391145462724
235,Albania,coal,0.024317314523581157,0.025770486274170428,0.029070074678547934,0.03146270929689194,0.03399786552216729,0.03624797855457778,0.03566855295477164,0.03728748741744574,...,0.0035249441657845544,0.0033113416915863596,0.0029688882665851292,0.002642520161491151,0.003339683613892591,0.002822255337070553,0.002199419837735592,0.002175237164280488,0.002180172995948474,0.0024943557431554015
236,Algeria,coal,0.0025473980424690033,0.007655064470875166,0.020475800605753863,0.03321382392874011,0.03423572313878258,0.03424213176739014,0.034610210701987144,0.035079318419173874,...,0.000046554578705383786,0.000054955601161265284,0.0004334178297892882,0.0012596214769158304,0.0008353278614203434,0.0005769222932545153,0.0005752187520015624,0.000924225066126768,0.00041041788252525746,0.00040543921847342187
237,American Samoa,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,Wake Island,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,Western Sahara,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
461,Yemen,coal,0,0,0,0,0,0,0,0,...,0.00437591677914764,0.00447887952689229,0.004581842274636941,0.0047877677701262415,0.004813508457062404,0.004015547162041363,0.003474992736381949,0.0032690672408926483,0.002372697626570008,0.0008509773029078125
462,Zambia,coal,0.015136894032273961,0.012094153254484357,0.013189697449913943,0.011776885530685234,0.011356318624572684,0.011589222073092067,0.012851651910673069,0.010866791206443746,...,0.003154689702533103,0.0017497830731440316,0.002530572674631151,0.0036506641640669996,0.0038833591803633235,0.004156763505309775,0.009056686670882155,0.020752247493247774,0.03101003454569231,0.03415060606327614


In [107]:
coal_prod

,Country,Energy_type,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
233,World,coal,80.11419429217284,80.46072857252325,83.39892921243354,83.68635266676536,87.66942309140777,91.67976259540721,94.04460314569837,96.27315618601719,...,154.18564329417617,164.50258655027486,168.87502547032878,171.4292379737608,170.5717525951305,166.93537676884293,157.08872352111496,161.47611704294215,167.39449030449197,170.91325027629023
234,Afghanistan,coal,0.0023552858074806604,0.0024740397484436876,0.002869886158469484,0.002869886158469484,0.0029292630391745574,0.002988640099432511,0.0031667709211006113,0.003305317155631996,...,0.014349430612794539,0.029292594481169477,0.024542444024763615,0.026660219436577894,0.030032826260626058,0.027016480720808336,0.03360731447907147,0.04336491520827184,0.04839215777463471,0.042335715942717234
235,Albania,coal,0.013229038633645784,0.014367221031874504,0.01585991934508173,0.01725932404545147,0.01875202227402413,0.020058133276921814,0.02021673252877048,0.019908863462528004,...,0.00009329352820292374,0.00004664676410146187,0.00006530546974204662,0.00008396417538263138,0.00015533371176268317,0.00022670324814273499,0.00013061093948409325,0.0001294802219222738,0.00010665580745664692,0.0000889843001352307
236,Algeria,coal,0.0000759200610490114,0.00024041352665520277,0.00040490699226139416,0.0005694004578675855,0.0005757271296216698,0.0005820538013757541,0.00027837355717970846,0.00024041352665520277,...,0,0,0,0,0,0,0,0,0,0
237,American Samoa,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,Wake Island,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,Western Sahara,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
461,Yemen,coal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
462,Zambia,coal,0.013940887643917078,0.012427445672371548,0.014806025973126137,0.01108993602526018,0.01257289063705352,0.01248247963956641,0.013500519425967932,0.011320486231343822,...,0,0,0.00220037170449094,0.003647752965039811,0.003893295613831119,0.004021876348058157,0.008067354364426004,0.019626050091859793,0.028916971158526542,0.03259616191934048


In [108]:
energy_consumption_per_capita

,Country,Energy_type,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,World,energy_consumption_per_capita,68.1459548678342,66.11779195371682,64.99939955926463,64.73001551716604,67.18740371473154,67.72669736181508,67.89798545851839,68.81176088259464,...,75.85021734387001,76.81301793921361,77.45018264495602,77.17074501779302,76.90143146117923,76.24857246626453,76.04634169493572,76.74242174383677,77.59177446964536,77.21384657054325
2,Afghanistan,energy_consumption_per_capita,1.9902831338720353,2.2737731366371663,2.476057805069556,3.11235738067043,3.212166236643484,3.206091479049969,3.332526591235134,5.434543297122344,...,4.061555520719247,5.347433782037638,4.736874741480483,3.8582527606273733,3.4255622019929817,3.6697740511736567,3.32294192971859,3.4422777418168047,3.8541735230994862,3.225900605229116
3,Albania,energy_consumption_per_capita,60.752906333637476,59.20471634727863,58.361926760819614,59.52323005683565,58.67682750151277,57.26436102552498,54.554686870223655,53.326846191744856,...,41.73771071625683,36.684247231169316,36.075733185359105,45.43411116658544,40.12567468285529,40.2340417763733,44.90231275511119,43.462449016341786,49.33449257004024,38.4553185271436
4,Algeria,energy_consumption_per_capita,40.61530286672366,33.46354339032917,46.5551641047492,50.73295974373409,51.95770520367143,46.64949943908242,46.155419424041966,47.88021745831313,...,49.413253929302826,52.861873082392634,56.125875710745674,53.42370884022648,59.08736571185966,59.19709604738706,57.609648342121254,57.04872044051388,59.60163688886581,61.433374033951914
5,American Samoa,energy_consumption_per_capita,180.5156037493108,205.52735082045044,276.1121453900709,167.16753405470723,183.03520529173036,164.64715073761988,163.34502298207642,163.1494822999532,...,153.47911874320155,88.08459434095352,88.4719067497301,88.15732227410527,88.03407172944071,88.000915373486,88.35445012791303,88.30472664183095,88.55149906821667,88.79644373406562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Wake Island,energy_consumption_per_capita,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228,Western Sahara,energy_consumption_per_capita,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229,Yemen,energy_consumption_per_capita,12.023856541260635,11.825640810375528,13.37893325948553,11.930130300453301,14.280471070455958,15.25478718668517,14.135172164046232,13.50001404766518,...,14.697440389980297,12.01045336498422,10.12405147950513,13.904941263896283,12.817469070957166,6.743885590814418,5.26422758031014,4.7417183975416615,4.307652600577925,5.452938019556406
230,Zambia,energy_consumption_per_capita,22.747916019233678,21.877840907717573,21.60736547262854,20.799905583861577,19.46846127888287,18.66665089468799,17.99726743613171,16.170192226795223,...,9.574452142182206,10.407155811456462,10.84024604027509,11.139359845692802,11.588748947578106,10.875946387762607,9.953336227194274,11.146889846813012,12.169115443677525,11.586566793571398


In [109]:
CO2_emissions

,Country,Energy_type,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,World,CO2_emissions,4946.62713,18701.988660053,18327.455974396,18284.018922293,18478.503517135,20118.811365854,20531.213799098,21064.157108403,...,31639.876165687,33609.984153055,34720.267922662,35537.365120578,35944.451742368,35976.723878385,35781.370769103,35117.963601548,35887.898987724,36638.369585101
2,Afghanistan,CO2_emissions,NaN,1.325964518,1.509627735,1.65960468,2.05960468,2.065436114,2.093515917,2.311010238,...,6.230759779,7.794436305,11.097938929,9.753422695,7.910700222,7.169380063,7.852509211,7.279395964,7.886124149,9.391163087
3,Albania,CO2_emissions,NaN,9.618526399,9.533068754,9.594264089,9.981384922,10.126808525,10.0889756,10.154939541,...,3.02339894,3.673558222,3.756730087,3.393630544,3.857129766,4.043419507,3.913785734,4.105771205,4.483725681,4.031246029
4,Algeria,CO2_emissions,NaN,45.663590904,39.376193382,56.531989464,63.168954306,66.766862302,61.757201027,60.918466391,...,99.380185852,101.352896736,112.488751213,122.016837732,118.720563943,131.695938497,138.622437629,137.190705524,137.678348102,145.987969135
5,American Samoa,CO2_emissions,NaN,0.425070708,0.499607404,0.696422442,0.439048931,0.500976192,0.469328168,0.484467786,...,0.601936254,0.601936254,0.35542766,0.356401475,0.35542766,0.35542766,0.35542766,0.355427692,0.35542766,0.35542766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Wake Island,CO2_emissions,NaN,1.23566,1.2,1.2,1.2,1.2,1.2,1.3,...,1.3,1.295447529,1.297896871,1.301452752,1.297896871,1.297896871,1.297896871,1.298108468,1.271954959,1.273786855
228,Western Sahara,CO2_emissions,NaN,0.15699,0.2,0.2,0.2,0.2,0.2,0.2,...,0.3,0.3,0.267664006,0.260806065,0.26892239,0.2684852,0.270403,0.269783,0.26303824,0.26303824
229,Yemen,CO2_emissions,NaN,6.43422,6.6,7.7,7,8.9,9.9,10,...,20.948010277,24.339585376,20.318221681,17.044172859,23.975684299,22.354079518,11.979031441,9.687529038,9.043446352,8.368931695
230,Zambia,CO2_emissions,NaN,3.62725204,3.130211794,3.231777522,3.099228588,2.860020567,2.781549986,2.999671558,...,2.364738092,2.294449236,2.446639655,2.930656509,3.170716329,3.611288509,3.940613316,4.228833733,5.687526839,7.09628982


#### Joining econ and eprods temporary datasets back into single datsets

In [110]:
econ = pd.concat([total_con, coal_con, natural_gas_con, petroleum_n_other_liquids_con, nuclear_con, renewables_n_other_con])

eprod = pd.concat([total_prod, coal_prod, natural_gas_prod, petroleum_n_other_liquids_prod, nuclear_prod, renewables_n_other_prod])

In [111]:
econ

,Country,Energy_type,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,World,all_energy_types,292.8999360117116,289.40151331082484,289.69127227568276,293.7314691613172,307.2346122721334,315.42743540651674,322.18517006014446,332.74655318366644,...,527.0687498431604,540.8952031029451,551.9303162477122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,all_energy_types,0.02658321667756184,0.029949457623843767,0.031897814673808554,0.03902180313163165,0.03920224040186807,0.03827496129519434,0.039111198580053776,0.06306515769145625,...,0.11853852864995157,0.16105080218714038,0.1476076485691699,0.12450427328434109,0.1143137511302674,0.12629013628746977,0.11757565429923286,0.12494125714475693,0.14327311946093876,0.12274842133951257
3,Albania,all_energy_types,0.16298182182124926,0.16204449273682853,0.16284611700217974,0.16933585149179003,0.1704057218202433,0.17005625500667076,0.16607537777033485,0.1666405283961216,...,0.12304402332284661,0.10743348644120246,0.1051279333181231,0.13193111766441915,0.11621639284070061,0.11629730243544255,0.1296073826057356,0.12535309157946248,0.14218447431148448,0.11074824093269141
4,Algeria,all_energy_types,0.7806951671133022,0.6633913232329025,0.952188115918025,1.0705618432163024,1.1307867129706235,1.0464182466677772,1.0663009617019872,1.1383186539191739,...,1.7777604019160995,1.9379902738228294,2.0982041249829453,2.037585597537122,2.299898896758712,2.351782231770593,2.3361518937806958,2.361200900056517,2.5157731727516484,2.6435026580305645
5,American Samoa,all_energy_types,0.0058931124,0.00692647725,0.0096550895,0.00608690425,0.0069454539,0.00650669075,0.006716584,0.0069730088735,...,0.0086069555,0.004911508895857226,0.004924965633037225,0.004911508895857226,0.004911508895857226,0.004911507088825,0.00492496540458,0.004911508895818637,0.004911508895818637,0.004911508895818637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Wake Island,renewables_n_other,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,Western Sahara,renewables_n_other,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,Yemen,renewables_n_other,0,0,0,0,0,0,0,0,...,0.000019512,0.000019432,0.000028548,0.000038164,0.00008559,0.00102509,0.001357104,0.001685979,0.004169632,0.004452
1622,Zambia,renewables_n_other,0.0842504416429,0.09095160935112,0.09276673989719,0.093613620726,0.091707448,0.092414884,0.09076501,0.081980147,...,0.099506171504,0.111397717312,0.1142021,0.122629646264,0.12864310304,0.119627497176,0.106216508816,0.113591182889,0.120469815456,0.12081104372


In [112]:
eprod

,Country,Energy_type,1980,1981,1982,1983,1984,1985,1986,1987,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,World,all_energy_types,296.33722758863627,291.2877731828766,290.1205665759991,293.0466469371524,308.92101366220777,316.46426154850144,326.7530652183027,334.3845057886132,...,518.2107838608919,533.8925086831691,545.3963111029857,553.2388229380201,561.949952486594,567.4371747031059,563.3786478855133,577.8623821917073,600.076613912069,610.9984170870961
2,Afghanistan,all_energy_types,0.07256115614667066,0.09272365463746368,0.09506786743187949,0.09971281973566948,0.09982734303917455,0.12096783009943252,0.12054976892110061,0.114835081155632,...,0.02692579367150454,0.04104421211242948,0.037492262916033615,0.040816563280627896,0.04478950651933605,0.04205799729038833,0.04950511782484147,0.05950054255404184,0.06317585556177092,0.05521030952760256
3,Albania,all_energy_types,0.155561620237156,0.15869138293903517,0.20594415566567068,0.23469322168485068,0.19102506227402413,0.19172993827692245,0.19659552252877113,0.20238806446252838,...,0.09849116932673144,0.07585689950134678,0.0840378737211778,0.10563648550534094,0.09296194121961915,0.10225470403895913,0.12535621607056832,0.08229649004849868,0.11552948572255144,0.08908229857345225
4,Algeria,all_energy_types,2.803017355026457,3.0375368604701833,3.224933778884667,3.6064004828010217,3.8591760033483915,3.907465537569512,3.9683247789434253,4.21808633368004,...,6.927837879172395,6.828492246382395,6.999409831111563,6.5104186447871735,6.561760333289481,6.696192522564449,6.910709728136037,6.763856727052585,6.688552187874529,6.429790182459667
5,American Samoa,all_energy_types,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Wake Island,renewables_n_other,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,Western Sahara,renewables_n_other,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,Yemen,renewables_n_other,0,0,0,0,0,0,0,0,...,0.000019512,0.000019432,0.000028548,0.000038164,0.00008559,0.00102509,0.001357104,0.001685979,0.004169632,0.004452
1622,Zambia,renewables_n_other,0.0935959096429,0.10114325335112,0.10363054789719,0.104463780726,0.10213452,0.102923844,0.10127397,0.087029907,...,0.101433951504,0.111087225312,0.116989704,0.126075766264,0.13288421904,0.120961589176,0.101470416816,0.114648902889,0.124130891456,0.12346557972


### Stacking the Year Columns into one Column for econ, eprod, gdp, pop, ei_gdp_pop, and co2_e datasets

In [113]:
def stack_df(df,id_vars, value_name, column_names):
    year_list = ['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
                '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
                '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

    # Specifying which year columns to stack
    year_cols = [x for x in df if x in year_list]
    # Stacking all the years
    df = pd.melt(df, id_vars=id_vars, value_vars=year_cols, value_name=value_name)
    # Changing the Column Names
    df.columns = column_names
    return df


data_con = stack_df(econ, ['Country', 'Energy_type'],'consumption', ['Country', 'Energy_type', 'Year', 'Energy_consumption'])
data_prod = stack_df(eprod, ['Country', 'Energy_type'],'consumption', ['Country', 'Energy_type', 'Year', 'Energy_production'])
data_gdp = stack_df(gdp, 'Country', 'GDP', ['Country', 'Year', 'GDP'])
data_pop = stack_df(pop, 'Country', 'Population', ['Country', 'Year', 'Population'])
data_ei_gdp = stack_df(energy_consumption_per_GDP, 'Country', 'Energy_intensity_by_GDP', ['Country', 'Year', 'Energy_intensity_by_GDP'])
data_ei_pop = stack_df(energy_consumption_per_capita, 'Country', 'Energy_intensity_per_capita', ['Country', 'Year', 'Energy_intensity_per_capita'])
data_co2 = stack_df(CO2_emissions, 'Country', 'CO2_emission', ['Country', 'Year', 'CO2_emission'])

In [114]:
data_con

,Country,Energy_type,Year,Energy_consumption
0,World,all_energy_types,1980,292.8999360117116
1,Afghanistan,all_energy_types,1980,0.02658321667756184
2,Albania,all_energy_types,1980,0.16298182182124926
3,Algeria,all_energy_types,1980,0.7806951671133022
4,American Samoa,all_energy_types,1980,0.0058931124
...,...,...,...,...
55435,Wake Island,renewables_n_other,2019,0
55436,Western Sahara,renewables_n_other,2019,0
55437,Yemen,renewables_n_other,2019,0.004452
55438,Zambia,renewables_n_other,2019,0.12081104372


In [115]:
data_prod

,Country,Energy_type,Year,Energy_production
0,World,all_energy_types,1980,296.33722758863627
1,Afghanistan,all_energy_types,1980,0.07256115614667066
2,Albania,all_energy_types,1980,0.155561620237156
3,Algeria,all_energy_types,1980,2.803017355026457
4,American Samoa,all_energy_types,1980,0
...,...,...,...,...
55435,Wake Island,renewables_n_other,2019,0
55436,Western Sahara,renewables_n_other,2019,0
55437,Yemen,renewables_n_other,2019,0.004452
55438,Zambia,renewables_n_other,2019,0.12346557972


In [116]:
data_gdp

,Country,Year,GDP
0,World,1980,27771.471791
1,Afghanistan,1980,NaN
2,Albania,1980,NaN
3,Algeria,1980,NaN
4,American Samoa,1980,NaN
...,...,...,...
9235,Wake Island,2019,NaN
9236,Western Sahara,2019,NaN
9237,Yemen,2019,53.0838
9238,Zambia,2019,247.1819


In [117]:
data_pop

,Country,Year,Population
0,World,1980,4298126.5224
1,Afghanistan,1980,13356.5
2,Albania,1980,2682.7
3,Algeria,1980,19221.7
4,American Samoa,1980,32.646
...,...,...,...
9235,Wake Island,2019,NaN
9236,Western Sahara,2019,NaN
9237,Yemen,2019,29162.2
9238,Zambia,2019,17873.85


In [118]:
data_ei_gdp

,Country,Year,Energy_intensity_by_GDP
0,World,1980,10.546791981929914
1,Afghanistan,1980,0
2,Albania,1980,0
3,Algeria,1980,0
4,American Samoa,1980,0
...,...,...,...
9235,Wake Island,2019,0
9236,Western Sahara,2019,0
9237,Yemen,2019,2.9956346213705087
9238,Zambia,2019,0.8378305890652841


In [119]:
data_ei_pop

,Country,Year,Energy_intensity_per_capita
0,World,1980,68.1459548678342
1,Afghanistan,1980,1.9902831338720353
2,Albania,1980,60.752906333637476
3,Algeria,1980,40.61530286672366
4,American Samoa,1980,180.5156037493108
...,...,...,...
9235,Wake Island,2019,0
9236,Western Sahara,2019,0
9237,Yemen,2019,5.452938019556406
9238,Zambia,2019,11.586566793571398


In [120]:
data_co2

,Country,Year,CO2_emission
0,World,1980,4946.62713
1,Afghanistan,1980,NaN
2,Albania,1980,NaN
3,Algeria,1980,NaN
4,American Samoa,1980,NaN
...,...,...,...
9235,Wake Island,2019,1.273786855
9236,Western Sahara,2019,0.26303824
9237,Yemen,2019,8.368931695
9238,Zambia,2019,7.09628982


#### Merging all datasets, data_con, data_prod, data_gdp, and data_pop into one dataset df

In [121]:
df = data_con.merge(data_prod)
df = df.merge(data_gdp)
df = df.merge(data_pop)
df = df.merge(data_ei_pop)
df = df.merge(data_ei_gdp)
df = df.merge(data_co2)

### !!!! Desired Columns So Far !!!
['Country', 'Energy_type', 'Year', 'Energy_consumption', 'Energy_production', 'GDP', 'Population', 'energy intensity by GDP and population', 'C02_emissions']

## Column descriptions
1. Country - Name of country
2. Energy_type - Type of energy
3. Year - Year documented
4. Energy_consumption - Amount of a specific energy source consumed each year, measured (quad Btu)
5. Energy_production - Amount of specific energy source produced each year, measured (quad Btu)
6. GDP -  Countries GDP at purchasing power parities, measured (Billion 2015$$ PPP)
7. Population - Countries Population, measured (Mperson)
8. Energy_intensity_by_GDP - Energy intensity is a measure of the energy inefficiency of an economy. It is calculated as units of energy per unit of GDP (1000 Btu/2015$ GDP PPP)
9. Energy_intensity_per_capita - Energy intensity is a measure of the energy inefficiency of an economy. It is calculated as units of energy per unit of capita (capita = individual person) (MMBtu/person)
10. CO2_emissions - CO2 emissions (MMtonnes CO2)

In [122]:
df

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,World,all_energy_types,1980,292.8999360117116,296.33722758863627,27771.471791,4298126.5224,68.1459548678342,10.546791981929914,4946.62713
1,World,coal,1980,78.65628042083658,80.11419429217284,27771.471791,4298126.5224,68.1459548678342,10.546791981929914,4946.62713
2,World,natural_gas,1980,53.865223303,54.761045594,27771.471791,4298126.5224,68.1459548678342,10.546791981929914,4946.62713
3,World,petroleum_n_other_liquids,1980,132.06401937163076,133.11110886607943,27771.471791,4298126.5224,68.1459548678342,10.546791981929914,4946.62713
4,World,nuclear,1980,7.575700462108056,7.575700462108056,27771.471791,4298126.5224,68.1459548678342,10.546791981929914,4946.62713
...,...,...,...,...,...,...,...,...,...,...
55435,Zimbabwe,coal,2019,0.04506367279138204,0.04776061221068363,37.6204,14654.2,11.547221559136965,4.497966373879728,8.966856508
55436,Zimbabwe,natural_gas,2019,0,0,37.6204,14654.2,11.547221559136965,4.497966373879728,8.966856508
55437,Zimbabwe,petroleum_n_other_liquids,2019,0.0554979345,0,37.6204,14654.2,11.547221559136965,4.497966373879728,8.966856508
55438,Zimbabwe,nuclear,2019,NaN,NaN,37.6204,14654.2,11.547221559136965,4.497966373879728,8.966856508


### Questions I want to answer

* what type of energy source do 3rd world countries use the most?
* what type of energy source do 1st world countries use the most?
* Who produces the most co2 emissions, 1st world or 3rd world countries?
* who produces the most clean energy?
* who produces the most dirty energy?
* who produces the most energy overall?
* for each country, how has gdp and population affected energy consumption, production and emission
* Are 1st world counties better at reducing emisions by energy_intensity_per_capita (per person) than 3rd world countries
* what role does capitalistic counties play in co2 emmisions with relations to gdp, (energy intensity per gdp), population and (energy intensity per capita)